In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import warnings
from scipy import stats
from scipy.stats import percentileofscore
from IPython.utils import io

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option("display.precision", 10)
stars_dir = '~/GitHub/stars-data-builder/'


# For 2025

In [2]:
with io.capture_output() as captured: 
    raw_data = pd.read_sas(stars_dir + '2025/2025-07 Stars Release/alldata_2025jul.sas7bdat')
raw_data = raw_data.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

raw_data.head()

,PROVIDER_ID,COMP_HIP_KNEE,IMM_3_DEN,EDAC_30_AMI,EDAC_30_AMI_DEN,READM_30_HOSP_WIDE_DEN,HAI_3,EDAC_30_HF_DEN,OP_18B_DEN,HAI_6,OP_29,OP_23_DEN,SAFE_USE_OF_OPIOIDS,MORT_30_COPD_DEN,HAI_3_DEN_VOL,HAI_5_DEN_PRED,OP_35_ED,OP_13_DEN,READM_30_CABG,HAI_5_DEN_VOL,OP_22,HAI_1_DEN_VOL,OP_36,MORT_30_STK,READM_30_COPD,READM_30_HIP_KNEE_DEN,H_NUMB_COMP,HAI_6_DEN_VOL,MORT_30_PN,OP_35_ADM,PC_01,MORT_30_HF_DEN,OP_32,PC_01_DEN,SAFE_USE_OF_OPIOIDS_DEN,H_COMP_7_STAR_RATING,READM_30_HIP_KNEE,HCP_COVID_19,READM_30_COPD_DEN,H_COMP_5_STAR_RATING,OP_10_DEN,COMP_HIP_KNEE_DEN,HAI_2_DEN_PRED,MORT_30_CABG,OP_29_DEN,OP_22_DEN,SEP_1,OP_23,MORT_30_AMI_DEN,H_COMP_6_STAR_RATING,OP_18B,MORT_30_AMI,MORT_30_PN_DEN,READM_30_HOSP_WIDE,EDAC_30_PN,OP_36_DEN,MORT_30_CABG_DEN,HAI_1_DEN_PRED,HAI_6_DEN_PRED,MORT_30_COPD,OP_13,PSI_90_SAFETY,MORT_30_HF,H_RESP_RATE_P,HCP_COVID_19_DEN,H_COMP_1_STAR_RATING,READM_30_CABG_DEN,PSI_90_SAFETY_DEN,IMM_3,HAI_3_DEN_PRED,H_COMP_3_STAR_RATING,OP_8_DEN,MORT_30_STK_DEN,OP_8,PSI_4_SURG_COMP_DEN,OP_10,EDAC_30_HF,HAI_2_DEN_VOL,OP_32_DEN,OP_35_ED_DEN,HAI_2,H_COMP_2_STAR_RATING,HAI_1,PSI_4_SURG_COMP,OP_35_ADM_DEN,SEP_1_DEN,EDAC_30_PN_DEN,HAI_5,HAI_4,HAI_4_DEN_VOL,HAI_4_DEN_PRED,H_INDI_STAR_RATING,H_GLOB_STAR_RATING
0,010001,0.030,4115.0,-13.8,296.0,2924.0,1.209,679.0,345.0,0.491,0.47,NaN,0.12,122.0,240.0,11.232,4.9,193.0,0.102,103195.0,0.05,9538.0,1.1,0.137,0.177,34.0,643.0,103195.0,0.189,11.9,0.02,610.0,12.9,46.0,4303.0,4.0,0.044,0.000,130.0,3.0,1478.0,32.0,23.712,0.038,17.0,52960.0,0.68,NaN,291.0,3.0,217.0,0.108,489.0,0.141,14.3,668.0,157.0,10.082,65.234,0.079,0.021,0.98,0.105,17.0,2712.0,2.0,151.0,NaN,0.96,6.618,3.0,66.0,414.0,0.333,125.0,0.054,10.6,16332.0,170.0,202.0,0.169,4.0,0.496,194.78,202.0,131.0,490.0,0.445,NaN,NaN,NaN,3.0,4.0
1,010005,0.027,2407.0,NaN,NaN,1056.0,0.000,176.0,1154.0,0.631,0.96,13.0,0.16,133.0,102.0,1.631,5.5,177.0,NaN,35424.0,0.03,3410.0,1.9,0.139,0.161,144.0,714.0,33872.0,0.231,7.9,0.01,162.0,14.2,193.0,2244.0,3.0,0.039,0.026,143.0,4.0,850.0,138.0,3.262,NaN,180.0,56820.0,0.76,0.69,28.0,4.0,144.0,0.145,301.0,0.135,-8.4,406.0,NaN,2.074,9.503,0.092,0.028,1.01,0.127,17.0,2431.0,3.0,NaN,NaN,0.72,2.867,2.0,109.0,97.0,0.459,43.0,0.134,9.8,7200.0,739.0,107.0,1.226,4.0,0.964,191.14,107.0,288.0,305.0,2.452,NaN,NaN,NaN,3.5,3.0
2,010006,0.040,2560.0,13.4,315.0,2560.0,0.000,508.0,349.0,0.026,0.85,NaN,0.16,141.0,73.0,5.397,NaN,243.0,0.115,68020.0,0.01,4826.0,1.4,0.142,0.177,90.0,1620.0,64728.0,0.198,NaN,0.00,455.0,12.1,22.0,3521.0,2.0,0.044,0.000,154.0,2.0,973.0,84.0,11.164,0.052,82.0,42286.0,0.56,NaN,292.0,2.0,177.0,0.166,616.0,0.147,4.5,484.0,102.0,5.279,37.771,0.102,0.029,1.16,0.122,18.0,2095.0,2.0,91.0,NaN,0.66,1.935,2.0,NaN,271.0,NaN,102.0,0.108,-4.9,8700.0,1355.0,NaN,0.000,2.0,0.000,201.13,NaN,162.0,621.0,0.371,0.0,116.0,1.086,2.5,2.0
3,010007,0.034,345.0,NaN,NaN,234.0,NaN,35.0,594.0,1.003,0.23,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,0.04,NaN,1.2,NaN,0.190,33.0,175.0,5267.0,0.290,NaN,NaN,33.0,13.4,NaN,NaN,4.0,0.044,0.000,41.0,5.0,151.0,33.0,NaN,NaN,111.0,11202.0,0.24,NaN,NaN,5.0,129.0,NaN,101.0,0.151,21.5,59.0,NaN,NaN,1.994,0.135,NaN,0.93,0.141,24.0,245.0,5.0,NaN,NaN,0.26,NaN,4.0,NaN,NaN,NaN,NaN,0.073,20.9,NaN,109.0,NaN,NaN,5.0,NaN,NaN,NaN,21.0,85.0,NaN,NaN,NaN,NaN,3.0,4.0
4,010008,NaN,126.0,NaN,NaN,81.0,NaN,NaN,329.0,NaN,0.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.192,NaN,NaN,NaN,12.9,NaN,NaN,NaN,NaN,0.019,NaN,NaN,74.0,NaN,NaN,NaN,24.0,6239.0,NaN,NaN,NaN,NaN,121.0,NaN,26.0,0.153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0,NaN,NaN,NaN,0.39,NaN,NaN,NaN,NaN,NaN,NaN,0.014,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:

raw_data.dropna(how='all', axis=0, inplace=True)
for c in list(raw_data):
    if c == 'PROVIDER_ID': 
        continue
    else:
        raw_data[c] = raw_data[c].astype(float)

print(raw_data.shape)

# Define the measures you're interested in
measures = ['MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF',
            'MORT_30_PN', 'MORT_30_STK', 'PSI_4_SURG_COMP', 'COMP_HIP_KNEE',
            'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 
            'PSI_90_SAFETY', 'EDAC_30_AMI', 'EDAC_30_HF',
            'EDAC_30_PN', 'OP_32', 'READM_30_CABG', 'READM_30_COPD',
            'READM_30_HIP_KNEE', 'READM_30_HOSP_WIDE', 'OP_35_ADM', 
            'OP_35_ED', 'OP_36', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 
            'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 
            'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 
            'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'HCP_COVID_19', 
            'IMM_3', 'OP_10', 'OP_13', 'OP_18B', #'OP_2', 
            'OP_22',
            'OP_23', 'OP_29', #'OP_3B', 
            'OP_8', 'PC_01', 'SEP_1',
            'SAFE_USE_OF_OPIOIDS',
           ]

print(len(measures), 'measures')
prvdrs = raw_data['PROVIDER_ID'].tolist()
raw_data = raw_data.filter(items=measures)
filtered_data = raw_data.dropna(axis=1, thresh=101)
filtered_measures = list(filtered_data)

excluded = [item for item in measures if item not in filtered_measures]
print('Excluded measure(s):', excluded)
filtered_data.dropna(how='all', subset=filtered_measures, axis=0, inplace=True)

print('Shape of filtered dataframe:', filtered_data.shape)
print('Final no. of measures:', filtered_data.shape[1])

filtered_data['PROVIDER_ID'] = prvdrs
filtered_data = filtered_data[filtered_data.columns[-1:].tolist() + filtered_data.columns[:-1].tolist()]


## Replace the imputed 666666 suffixes of VHA hospitals with their original 'F' suffix
prvdrs1 = []
for p in filtered_data['PROVIDER_ID'].tolist():
    p = str(p)
    if '666666' in p:
        p = p[:-6] + 'F'
    while len(p) < 6:
        p = '0' + p
    prvdrs1.append(p)
    
filtered_data['PROVIDER_ID'] = prvdrs1

filtered_data.to_pickle(stars_dir + 'FilesForApp/2025_data_for_whatifs.pkl')
filtered_data.head()

(4566, 93)
46 measures
Excluded measure(s): []
Shape of filtered dataframe: (4566, 46)
Final no. of measures: 46


,PROVIDER_ID,MORT_30_AMI,MORT_30_CABG,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_4_SURG_COMP,COMP_HIP_KNEE,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,PSI_90_SAFETY,EDAC_30_AMI,EDAC_30_HF,EDAC_30_PN,OP_32,READM_30_CABG,READM_30_COPD,READM_30_HIP_KNEE,READM_30_HOSP_WIDE,OP_35_ADM,OP_35_ED,OP_36,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,HCP_COVID_19,IMM_3,OP_10,OP_13,OP_18B,OP_22,OP_23,OP_29,OP_8,PC_01,SEP_1,SAFE_USE_OF_OPIOIDS
0,010001,0.108,0.038,0.079,0.105,0.189,0.137,194.78,0.030,0.496,0.169,1.209,NaN,0.445,0.491,0.98,-13.8,10.6,14.3,12.9,0.102,0.177,0.044,0.141,11.9,4.9,1.1,2.0,4.0,3.0,3.0,3.0,4.0,4.0,3.0,0.000,0.96,0.054,0.021,217.0,0.05,NaN,0.47,0.333,0.02,0.68,0.12
1,010005,0.145,NaN,0.092,0.127,0.231,0.139,191.14,0.027,0.964,1.226,0.000,NaN,2.452,0.631,1.01,NaN,9.8,-8.4,14.2,NaN,0.161,0.039,0.135,7.9,5.5,1.9,3.0,4.0,2.0,4.0,4.0,3.0,3.0,3.5,0.026,0.72,0.134,0.028,144.0,0.03,0.69,0.96,0.459,0.01,0.76,0.16
2,010006,0.166,0.052,0.102,0.122,0.198,0.142,201.13,0.040,0.000,0.000,0.000,0.0,0.371,0.026,1.16,13.4,-4.9,4.5,12.1,0.115,0.177,0.044,0.147,NaN,NaN,1.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.5,0.000,0.66,0.108,0.029,177.0,0.01,NaN,0.85,NaN,0.00,0.56,0.16
3,010007,NaN,NaN,0.135,0.141,0.290,NaN,NaN,0.034,NaN,NaN,NaN,NaN,NaN,1.003,0.93,NaN,20.9,21.5,13.4,NaN,0.190,0.044,0.151,NaN,NaN,1.2,5.0,5.0,4.0,5.0,5.0,4.0,4.0,3.0,0.000,0.26,0.073,NaN,129.0,0.04,NaN,0.23,NaN,NaN,0.24,NaN
4,010008,NaN,NaN,NaN,NaN,0.192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.9,NaN,NaN,NaN,0.153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019,0.39,0.014,NaN,121.0,0.00,NaN,0.67,NaN,NaN,NaN,NaN


#
# For 2026 (predicted)

In [4]:
raw_data = pd.read_csv(stars_dir + "Reproduce_Stars_Input/2026/Input_File/data_for_2026_prognostications_from_Nov2025.csv")
raw_data.head()


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN,MORT_30_AMI,MORT_30_AMI_DEN,MORT_30_CABG,MORT_30_CABG_DEN,MORT_30_COPD,MORT_30_COPD_DEN,MORT_30_HF,MORT_30_HF_DEN,MORT_30_PN,MORT_30_PN_DEN,MORT_30_STK,MORT_30_STK_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,COMP_HIP_KNEE,COMP_HIP_KNEE_DEN,PSI_90_SAFETY,PSI_90_SAFETY_DEN,Hybrid_HWM,Hybrid_HWM_DEN,H_COMP_1_A_P,H_COMP_1_SN_P,H_COMP_1_U_P,H_COMP_1_LINEAR_SCORE,H_COMP_1_STAR_RATING,H_NURSE_RESPECT_A_P,H_NURSE_RESPECT_SN_P,H_NURSE_RESPECT_U_P,H_NURSE_LISTEN_A_P,H_NURSE_LISTEN_SN_P,H_NURSE_LISTEN_U_P,H_NURSE_EXPLAIN_A_P,H_NURSE_EXPLAIN_SN_P,H_NURSE_EXPLAIN_U_P,H_COMP_2_A_P,H_COMP_2_SN_P,H_COMP_2_U_P,H_COMP_2_LINEAR_SCORE,H_COMP_2_STAR_RATING,H_DOCTOR_RESPECT_A_P,H_DOCTOR_RESPECT_SN_P,H_DOCTOR_RESPECT_U_P,H_DOCTOR_LISTEN_A_P,H_DOCTOR_LISTEN_SN_P,H_DOCTOR_LISTEN_U_P,H_DOCTOR_EXPLAIN_A_P,H_DOCTOR_EXPLAIN_SN_P,H_DOCTOR_EXPLAIN_U_P,H_COMP_3_A_P,H_COMP_3_SN_P,H_COMP_3_U_P,H_COMP_3_LINEAR_SCORE,H_COMP_3_STAR_RATING,H_CALL_BUTTON_A_P,H_CALL_BUTTON_SN_P,H_CALL_BUTTON_U_P,H_BATH_HELP_A_P,H_BATH_HELP_SN_P,H_BATH_HELP_U_P,H_COMP_5_A_P,H_COMP_5_SN_P,H_COMP_5_U_P,H_COMP_5_LINEAR_SCORE,H_COMP_5_STAR_RATING,H_MED_FOR_A_P,H_MED_FOR_SN_P,H_MED_FOR_U_P,H_SIDE_EFFECTS_A_P,H_SIDE_EFFECTS_SN_P,H_SIDE_EFFECTS_U_P,H_COMP_6_N_P,H_COMP_6_Y_P,H_COMP_6_LINEAR_SCORE,H_COMP_6_STAR_RATING,H_DISCH_HELP_N_P,H_DISCH_HELP_Y_P,H_SYMPTOMS_N_P,H_SYMPTOMS_Y_P,H_COMP_7_A,H_COMP_7_D_SD,H_COMP_7_SA,H_COMP_7_LINEAR_SCORE,H_COMP_7_STAR_RATING,H_CT_PREFER_A,H_CT_PREFER_D_SD,H_CT_PREFER_SA,H_CT_UNDER_A,H_CT_UNDER_D_SD,H_CT_UNDER_SA,H_CT_MED_A,H_CT_MED_D_SD,H_CT_MED_SA,H_CLEAN_HSP_A_P,H_CLEAN_HSP_SN_P,H_CLEAN_HSP_U_P,H_CLEAN_LINEAR_SCORE,H_CLEAN_STAR_RATING,H_QUIET_HSP_A_P,H_QUIET_HSP_SN_P,H_QUIET_HSP_U_P,H_QUIET_LINEAR_SCORE,H_QUIET_STAR_RATING,H_HSP_RATING_0_6,H_HSP_RATING_7_8,H_HSP_RATING_9_10,H_HSP_RATING_LINEAR_SCORE,H_HSP_RATING_STAR_RATING,H_RECMND_DN,H_RECMND_DY,H_RECMND_PY,H_RECMND_LINEAR_SCORE,H_RECMND_STAR_RATING,H_STAR_RATING,O_COMP_1_LINEAR_SCORE,O_COMP_2_LINEAR_SCORE,O_COMP_3_LINEAR_SCORE,O_PATIENT_RATE_LINEAR_SCORE,O_PATIENT_REC_LINEAR_SCORE,EDAC_30_AMI,EDAC_30_AMI_DEN,EDAC_30_HF,EDAC_30_HF_DEN,EDAC_30_PN,EDAC_30_PN_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,READM_30_CABG,READM_30_CABG_DEN,READM_30_COPD,READM_30_COPD_DEN,READM_30_HIP_KNEE,READM_30_HIP_KNEE_DEN,Hybrid_HWR,Hybrid_HWR_DEN,HAI_1_DEN_PRED,HAI_1_DEN_VOL,HAI_1,HAI_2_DEN_PRED,HAI_2_DEN_VOL,HAI_2,HAI_3_DEN_PRED,HAI_3_DEN_VOL,HAI_3,HAI_4_DEN_PRED,HAI_4_DEN_VOL,HAI_4,HAI_5_DEN_PRED,HAI_5_DEN_VOL,HAI_5,HAI_6_DEN_PRED,HAI_6_DEN_VOL,HAI_6,OP_13,OP_8,OP_10
0,10001,0.93,4625.0,222.0,387.0,0.05,57084.0,0.91,11.0,0.72,29.0,0.69,150.0,0.14,4583.0,0.114,270.0,0.030,144.0,0.094,112.0,0.102,583.0,0.184,517.0,0.135,395.0,203.00,118.0,0.032,27.0,0.95,NaN,0.045,1835.0,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,88.0,NaN,NaN,NaN,NaN,89.0,NaN,NaN,NaN,NaN,91.0,NaN,NaN,98.0,93.0,97.0,94.0,94.0,-15.6,273.0,-1.1,652.0,17.4,507.0,12.8,218.0,8.5,261.0,5.7,261.0,0.8,647.0,0.101,137.0,0.180,122.0,0.048,25.0,0.151,2824.0,9.440,8935.0,0.530,23.350,16255.0,0.086,6.562,229.0,0.457,0.903,97.0,NaN,10.937,109019.0,0.183,68.076,109019.0,0.382,0.038,0.308,0.053
1,10005,0.59,2856.0,137.0,1120.0,0.03,58624.0,0.27,11.0,1.00,210.0,0.75,289.0,0.15,1859.0,NaN,NaN,NaN,NaN,0.089,126.0,0.141,158.0,0.212,285.0,0.129,89.0,184.79,27.0,0.030,104.0,0.97,NaN,0.046,698.0,NaN,NaN,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN,NaN,

In [5]:
df_dupes = raw_data[raw_data.duplicated('PROVIDER_ID', keep=False)]

print(df_dupes.shape)
df_dupes.head()

(0, 176)


,PROVIDER_ID,IMM_3,IMM_3_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,SEP_1,SEP_1_DEN,SAFE_USE_OF_OPIOIDS,SAFE_USE_OF_OPIOIDS_DEN,MORT_30_AMI,MORT_30_AMI_DEN,MORT_30_CABG,MORT_30_CABG_DEN,MORT_30_COPD,MORT_30_COPD_DEN,MORT_30_HF,MORT_30_HF_DEN,MORT_30_PN,MORT_30_PN_DEN,MORT_30_STK,MORT_30_STK_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,COMP_HIP_KNEE,COMP_HIP_KNEE_DEN,PSI_90_SAFETY,PSI_90_SAFETY_DEN,Hybrid_HWM,Hybrid_HWM_DEN,H_COMP_1_A_P,H_COMP_1_SN_P,H_COMP_1_U_P,H_COMP_1_LINEAR_SCORE,H_COMP_1_STAR_RATING,H_NURSE_RESPECT_A_P,H_NURSE_RESPECT_SN_P,H_NURSE_RESPECT_U_P,H_NURSE_LISTEN_A_P,H_NURSE_LISTEN_SN_P,H_NURSE_LISTEN_U_P,H_NURSE_EXPLAIN_A_P,H_NURSE_EXPLAIN_SN_P,H_NURSE_EXPLAIN_U_P,H_COMP_2_A_P,H_COMP_2_SN_P,H_COMP_2_U_P,H_COMP_2_LINEAR_SCORE,H_COMP_2_STAR_RATING,H_DOCTOR_RESPECT_A_P,H_DOCTOR_RESPECT_SN_P,H_DOCTOR_RESPECT_U_P,H_DOCTOR_LISTEN_A_P,H_DOCTOR_LISTEN_SN_P,H_DOCTOR_LISTEN_U_P,H_DOCTOR_EXPLAIN_A_P,H_DOCTOR_EXPLAIN_SN_P,H_DOCTOR_EXPLAIN_U_P,H_COMP_3_A_P,H_COMP_3_SN_P,H_COMP_3_U_P,H_COMP_3_LINEAR_SCORE,H_COMP_3_STAR_RATING,H_CALL_BUTTON_A_P,H_CALL_BUTTON_SN_P,H_CALL_BUTTON_U_P,H_BATH_HELP_A_P,H_BATH_HELP_SN_P,H_BATH_HELP_U_P,H_COMP_5_A_P,H_COMP_5_SN_P,H_COMP_5_U_P,H_COMP_5_LINEAR_SCORE,H_COMP_5_STAR_RATING,H_MED_FOR_A_P,H_MED_FOR_SN_P,H_MED_FOR_U_P,H_SIDE_EFFECTS_A_P,H_SIDE_EFFECTS_SN_P,H_SIDE_EFFECTS_U_P,H_COMP_6_N_P,H_COMP_6_Y_P,H_COMP_6_LINEAR_SCORE,H_COMP_6_STAR_RATING,H_DISCH_HELP_N_P,H_DISCH_HELP_Y_P,H_SYMPTOMS_N_P,H_SYMPTOMS_Y_P,H_COMP_7_A,H_COMP_7_D_SD,H_COMP_7_SA,H_COMP_7_LINEAR_SCORE,H_COMP_7_STAR_RATING,H_CT_PREFER_A,H_CT_PREFER_D_SD,H_CT_PREFER_SA,H_CT_UNDER_A,H_CT_UNDER_D_SD,H_CT_UNDER_SA,H_CT_MED_A,H_CT_MED_D_SD,H_CT_MED_SA,H_CLEAN_HSP_A_P,H_CLEAN_HSP_SN_P,H_CLEAN_HSP_U_P,H_CLEAN_LINEAR_SCORE,H_CLEAN_STAR_RATING,H_QUIET_HSP_A_P,H_QUIET_HSP_SN_P,H_QUIET_HSP_U_P,H_QUIET_LINEAR_SCORE,H_QUIET_STAR_RATING,H_HSP_RATING_0_6,H_HSP_RATING_7_8,H_HSP_RATING_9_10,H_HSP_RATING_LINEAR_SCORE,H_HSP_RATING_STAR_RATING,H_RECMND_DN,H_RECMND_DY,H_RECMND_PY,H_RECMND_LINEAR_SCORE,H_RECMND_STAR_RATING,H_STAR_RATING,O_COMP_1_LINEAR_SCORE,O_COMP_2_LINEAR_SCORE,O_COMP_3_LINEAR_SCORE,O_PATIENT_RATE_LINEAR_SCORE,O_PATIENT_REC_LINEAR_SCORE,EDAC_30_AMI,EDAC_30_AMI_DEN,EDAC_30_HF,EDAC_30_HF_DEN,EDAC_30_PN,EDAC_30_PN_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN,READM_30_CABG,READM_30_CABG_DEN,READM_30_COPD,READM_30_COPD_DEN,READM_30_HIP_KNEE,READM_30_HIP_KNEE_DEN,Hybrid_HWR,Hybrid_HWR_DEN,HAI_1_DEN_PRED,HAI_1_DEN_VOL,HAI_1,HAI_2_DEN_PRED,HAI_2_DEN_VOL,HAI_2,HAI_3_DEN_PRED,HAI_3_DEN_VOL,HAI_3,HAI_4_DEN_PRED,HAI_4_DEN_VOL,HAI_4,HAI_5_DEN_PRED,HAI_5_DEN_VOL,HAI_5,HAI_6_DEN_PRED,HAI_6_DEN_VOL,HAI_6,OP_13,OP_8,OP_10


In [6]:
raw_data = pd.read_csv(stars_dir + "Reproduce_Stars_Input/2026/Input_File/data_for_2026_prognostications_from_Nov2025.csv")
print("raw_data.shape:", raw_data.shape)

# Ensure PROVIDER_ID is string and strip column names
raw_data['PROVIDER_ID'] = raw_data['PROVIDER_ID'].astype(str)
raw_data = raw_data.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
raw_data.columns = raw_data.columns.str.strip()
print("raw_data.shape:", raw_data.shape)

# Drop rows that are completely empty
raw_data.dropna(how='all', inplace=True)
print("raw_data.shape:", raw_data.shape)


# Convert all measure columns to float (skip PROVIDER_ID)
for c in raw_data.columns:
    if c != 'PROVIDER_ID':
        raw_data[c] = raw_data[c].astype(float)


# Define the measures of interest
measures = [
    'MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF',
    'MORT_30_PN', 'MORT_30_STK', 'PSI_4_SURG_COMP', 'COMP_HIP_KNEE',
    'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'PSI_90_SAFETY',
    'EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'OP_32', 'READM_30_CABG',
    'READM_30_COPD', 'READM_30_HIP_KNEE', 'Hybrid_HWR', 'Hybrid_HWM', 
    'OP_35_ADM', 'OP_35_ED', 'OP_36', 
    'IMM_3', 'OP_10', 'OP_13', 'OP_18B', 'OP_22', 'OP_23', 'OP_29', 'OP_8',
    'SEP_1', 'SAFE_USE_OF_OPIOIDS',
    'O_COMP_1_LINEAR_SCORE',
    'O_COMP_2_LINEAR_SCORE',
    'O_COMP_3_LINEAR_SCORE',
    'O_PATIENT_RATE_LINEAR_SCORE',
    'O_PATIENT_REC_LINEAR_SCORE',
    'H_COMP_1_LINEAR_SCORE',
    'H_COMP_2_LINEAR_SCORE', 
    'H_COMP_3_LINEAR_SCORE', 
    'H_COMP_5_LINEAR_SCORE', 
    'H_COMP_6_LINEAR_SCORE', 
    'H_COMP_7_LINEAR_SCORE', 
    'H_CLEAN_LINEAR_SCORE',  
    'H_QUIET_LINEAR_SCORE', 
    'H_RECMND_LINEAR_SCORE', 
    'H_HSP_RATING_LINEAR_SCORE',
]

print(len(measures), 'measures defined')

# Filter measures that meet minimum non-NA threshold
filtered_data = raw_data[['PROVIDER_ID'] + measures]

'''
print("filtered_data.shape:", filtered_data.shape)
filtered_data = filtered_data.dropna(axis=1, thresh=101)  # keep columns with at least 101 non-NA
print("filtered_data.shape:", filtered_data.shape)

filtered_measures = [c for c in filtered_data.columns if c != 'PROVIDER_ID']
excluded = [m for m in measures if m not in filtered_measures]
print('Excluded measure(s):', excluded)
print("filtered_data.shape:", filtered_data.shape)

# Drop rows with all NA in the filtered measures
filtered_data.dropna(axis=0, how='all', subset=filtered_measures, inplace=True)
print('Shape after dropping all-NA rows:', filtered_data.shape)

# Reorder columns: PROVIDER_ID first
filtered_data = filtered_data[['PROVIDER_ID'] + filtered_measures]
'''

# Fix VHA 666666 suffixes and pad PROVIDER_ID to 6 characters
def fix_provider_id(p):
    p = str(p)
    if '666666' in p:
        p = p[:-6] + 'F'
    return p.zfill(6)

filtered_data['PROVIDER_ID'] = filtered_data['PROVIDER_ID'].apply(fix_provider_id)
print("filtered_data.shape:", filtered_data.shape)

# Save final dataframe as pickle for the app
filtered_data.to_pickle(stars_dir + 'FilesForApp/2026_data_for_whatifs.pkl')

# Optional: show first few rows
filtered_data.head()


raw_data.shape: (4791, 176)
raw_data.shape: (4791, 176)
raw_data.shape: (4791, 176)
52 measures defined
filtered_data.shape: (4791, 53)


,PROVIDER_ID,MORT_30_AMI,MORT_30_CABG,MORT_30_COPD,MORT_30_HF,MORT_30_PN,MORT_30_STK,PSI_4_SURG_COMP,COMP_HIP_KNEE,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,PSI_90_SAFETY,EDAC_30_AMI,EDAC_30_HF,EDAC_30_PN,OP_32,READM_30_CABG,READM_30_COPD,READM_30_HIP_KNEE,Hybrid_HWR,Hybrid_HWM,OP_35_ADM,OP_35_ED,OP_36,IMM_3,OP_10,OP_13,OP_18B,OP_22,OP_23,OP_29,OP_8,SEP_1,SAFE_USE_OF_OPIOIDS,O_COMP_1_LINEAR_SCORE,O_COMP_2_LINEAR_SCORE,O_COMP_3_LINEAR_SCORE,O_PATIENT_RATE_LINEAR_SCORE,O_PATIENT_REC_LINEAR_SCORE,H_COMP_1_LINEAR_SCORE,H_COMP_2_LINEAR_SCORE,H_COMP_3_LINEAR_SCORE,H_COMP_5_LINEAR_SCORE,H_COMP_6_LINEAR_SCORE,H_COMP_7_LINEAR_SCORE,H_CLEAN_LINEAR_SCORE,H_QUIET_LINEAR_SCORE,H_RECMND_LINEAR_SCORE,H_HSP_RATING_LINEAR_SCORE
0,010001,0.114,0.030,0.094,0.102,0.184,0.135,203.00,0.032,0.530,0.086,0.457,NaN,0.183,0.382,0.95,-15.6,-1.1,17.4,12.8,0.101,0.180,0.048,0.151,0.045,8.5,5.7,0.8,0.93,0.053,0.038,222.0,0.05,0.91,0.72,0.308,0.69,0.14,98.0,93.0,97.0,94.0,94.0,90.0,92.0,82.0,79.0,87.0,82.0,85.0,88.0,91.0,89.0
1,010005,NaN,NaN,0.089,0.141,0.212,0.129,184.79,0.030,0.728,1.003,0.436,NaN,1.590,0.530,0.97,NaN,12.2,-17.2,14.2,NaN,0.171,0.042,0.133,0.046,8.6,6.2,1.0,0.59,0.128,0.033,137.0,0.03,0.27,1.00,0.422,0.75,0.15,98.0,96.0,98.0,94.0,93.0,91.0,93.0,77.0,72.0,88.0,80.0,85.0,87.0,85.0,87.0
2,010006,0.145,0.054,0.087,0.125,0.196,0.124,236.12,0.047,0.000,0.000,0.000,0.0,0.000,0.078,1.14,-18.0,-4.7,-0.9,11.6,0.106,0.191,0.051,0.159,0.052,NaN,NaN,1.0,0.64,0.088,0.041,150.0,0.01,0.67,0.86,NaN,0.66,0.15,98.0,94.0,97.0,93.0,91.0,89.0,88.0,76.0,70.0,82.0,77.0,79.0,84.0,80.0,83.0
3,010007,NaN,NaN,0.112,0.134,0.254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.675,1.06,NaN,77.9,29.8,12.5,NaN,0.186,NaN,0.154,0.048,NaN,NaN,NaN,0.29,0.059,NaN,117.0,0.02,NaN,0.65,NaN,0.13,0.15,97.0,92.0,96.0,91.0,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,010008,NaN,NaN,NaN,NaN,0.150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.9,12.8,NaN,NaN,NaN,0.147,0.043,NaN,NaN,NaN,0.31,0.019,NaN,112.0,0.01,NaN,0.67,NaN,NaN,0.18,98.0,96.0,97.0,93.0,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
prvdrs1 = filtered_data['PROVIDER_ID'].tolist()
print(len(prvdrs1), 'hospitals')
print(len(list(set(prvdrs1))), 'hospitals')


4791 hospitals
4791 hospitals
